In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.sparse as sp
from torch.autograd import Variable

In [0]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def load_data(path="./data/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges_ordered_flatten = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj_unnormalize = adj.copy() # copy the unnormalized adjacency matrix

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(270)
    idx_val = range(270, 2707)
    idx_test = range(2200, 2707)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test, adj_unnormalize

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [3]:
# Load data
# I use Kipf's load_data() and modified it by returning an unnormalized adjacency matrix
adj, features, labels, idx_train, idx_val, idx_test, adj_unnormalize = load_data('')
adj_array = adj_unnormalize.toarray() # transform csr to dense array

Loading cora dataset...


In [0]:
# Build modularity matrix. For this part, please refer to Section 2.2 of [1]
# k_i: the degree of vertex i; total number of edges: m = 1/2 * sum_i(k_i)
# (k_i*k_j)/(2*m) describes the expected number of edges between vertices i and j if edges are placed randomly
# Modularity matrix b_ij = a_ij - (k_i*k_j)/(2*m)
degrees = np.array(adj_array.sum(1)) # calculate the degree of each node
degrees = degrees[:,np.newaxis] # shape: V * 1
m = 1/2 * degrees.sum() # the total number of edges in the network
B = adj_array - 1/(2*m) * degrees.dot(degrees.T) # Modularity matrix
B_tensor = torch.from_numpy(B) # convert numpy array into torch tensor

In [0]:
# This loss function adds the AutoEncoder loss and the pairwise loss together.
# lamd is a tradeoff hyperparameter.
class Semi_Loss(torch.nn.Module):
    
    def __init__(self, lamd=0.01):
        super(Semi_Loss,self).__init__()
        self.lamd = lamd
        
    def forward(self, AE_loss, semi_loss):
        semi_loss = AE_loss + self.lamd * semi_loss
        return semi_loss

In [0]:
# build Laplace matrix. For this part, please refer to Section 4 of [1]
# L = D - O
labels_train = encode_onehot(labels[idx_train].numpy())
n = idx_train.numpy().size
O = labels_train.dot(labels_train.T) - np.eye(n) # O denotes pairwise constraint matrix
L = np.diag(O.sum(1)) - O # L = D - O, D is a diagonal matrix whose entries are row summation of O
L = torch.from_numpy(L).float() # convert numpy array into torch float tensor

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
# calculate normalized mutual info to measure the clustering performance
def cal_nmi(data, labels, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++').fit(data)
    pred_tr = kmeans.labels_
    nmi = normalized_mutual_info_score(labels, pred_tr, average_method='arithmetic')
    return nmi

In [0]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, output_size):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(output_size, input_size),
        )

    def forward(self, x):
        # Train each autoencoder individually
        x = x.detach()
        y = self.encoder(x)
            
        return y.detach(), y 

    def reconstruct(self, x):
        return self.decoder(x)

class StackedAutoEncoder(nn.Module):
    r"""
    A stacked autoencoder made from the convolutional denoising autoencoders above.
    Each autoencoder is trained independently and at the same time.
    """

    def __init__(self):
        super(StackedAutoEncoder, self).__init__()

        self.AE1 = AutoEncoder(2708, 512)
        self.AE2 = AutoEncoder(512, 256)
        self.AE3 = AutoEncoder(256, 128)

    def forward(self, x):
        a1,h1 = self.AE1(x) # a1 has been detached. h1 can be used for backpropagation
        a2,h2 = self.AE2(a1)
        a3,h3 = self.AE3(a2)
        
        return h1, h2, h3, self.reconstruct(a3)

    def reconstruct(self, x):
            a2_reconstruct = self.AE3.reconstruct(x)
            a1_reconstruct = self.AE2.reconstruct(a2_reconstruct)
            x_reconstruct = self.AE1.reconstruct(a1_reconstruct)
            return x_reconstruct, a1_reconstruct, a2_reconstruct

In [0]:
B_tensor = B_tensor.cuda()
L = L.cuda()
labels = labels.cpu()

In [0]:
lamd = 5*10**-8
model = StackedAutoEncoder()
model = model.cuda()
labels = labels.cpu()
criterion = Semi_Loss(lamd)
criterion_AE = nn.MSELoss()
# criterion_AE = CrossEntropy_Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=10**-9)#10**-6
epochs = 1000
totalloss_list=[];pairwise_list=[];AE_list=[];
nmi_train1=[]; nmi_train2=[]; nmi_train3=[];
nmi_val1=[]; nmi_val2=[]; nmi_val3=[];

for epoch in range(epochs):
    model.train()
    B_tensor = Variable(B_tensor).cuda()
    H1, H2, H3, M = model(B_tensor) # H denotes low-embedding vectors, M is the reconstruction matrix
    # For pairwise_loss, we only need to compute those which have labels.
#     Our goal is to learn the low embedding representations of all vertices via a small portion of vertices with labels
    pairwise1 = torch.mm( torch.mm(H1[idx_train,:].t(), L), H1[idx_train,:] )# H'*L*H
    pairwise2 = torch.mm( torch.mm(H2[idx_train,:].t(), L), H2[idx_train,:] )
    pairwise3 = torch.mm( torch.mm(H3[idx_train,:].t(), L), H3[idx_train,:] )
    pairwise_loss = torch.trace(pairwise1) + torch.trace(pairwise2) + torch.trace(pairwise3)# trace(H'*L*H)
#     pairwise_loss = torch.trace(pairwise3)
    AE_loss = criterion_AE(M[0], B_tensor) + criterion_AE(M[1], H1) + criterion_AE(M[2], H2)# compute the reconstruction loss of AE
    total_loss = criterion(AE_loss, pairwise_loss) # total loss= recon_loss + lamd*pairwise_loss
    totalloss_list.append(total_loss.item()); pairwise_list.append(pairwise_loss.item()); AE_list.append(AE_loss.item())
    if epoch%10 == 0:
        model.eval()
        H1, H2, H3, M = model(B_tensor)
        H1, H2, H3 = H1.cpu(), H2.cpu(), H3.cpu()
        nmi_train_H1 = cal_nmi(data=H1[idx_train.numpy(),:].detach().numpy(), labels=labels[idx_train.numpy()], n_clusters=7)
        nmi_train_H2 = cal_nmi(data=H2[idx_train.numpy(),:].detach().numpy(), labels=labels[idx_train.numpy()], n_clusters=7)
        nmi_train_H3 = cal_nmi(data=H3[idx_train.numpy(),:].detach().numpy(), labels=labels[idx_train.numpy()], n_clusters=7)
        nmi_val_H1 = cal_nmi(data=H1[idx_val.numpy(),:].detach().numpy(), labels=labels[idx_val.numpy()], n_clusters=7)
        nmi_val_H2 = cal_nmi(data=H2[idx_val.numpy(),:].detach().numpy(), labels=labels[idx_val.numpy()], n_clusters=7)
        nmi_val_H3 = cal_nmi(data=H3[idx_val.numpy(),:].detach().numpy(), labels=labels[idx_val.numpy()], n_clusters=7)
        print('epoch{}:total_loss:{:.6f}|AE_loss: {:.6f}|pair_loss: {:.6f}|NMI train(H1 H2 H3): '
              '{:.2f} {:.2f} {:.2f}|val: {:.2f} {:.2f} {:.2f}'.format
              (epoch, total_loss, AE_loss, pairwise_loss, nmi_train_H1, nmi_train_H2, nmi_train_H3, nmi_val_H1,
              nmi_val_H2, nmi_val_H3))
        model.train()
        nmi_train1.append(nmi_train_H1); nmi_train2.append(nmi_train_H2); nmi_train3.append(nmi_train_H3)
        nmi_val1.append(nmi_val_H1); nmi_val2.append(nmi_val_H2); nmi_val3.append(nmi_val_H3)
        
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [0]:
np.savez('im22-data',epochs=epochs,totalloss_list=totalloss_list,AE_list=AE_list,pairwise_list=pairwise_list,
         nmi_train1=nmi_train1,nmi_train2=nmi_train2,nmi_train3=nmi_train3,
         nmi_val1=nmi_val1,nmi_val2=nmi_val2,nmi_val3=nmi_val3)

In [0]:
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt
host = host_subplot(111)
par = host.twinx()
host.set_xlabel("Epochs")
host.set_ylabel("Total-loss")
par.set_ylabel("Pairwise-loss")
p1, = host.plot(range(epochs), totalloss_list, 'r-', lw=2, label="Total-loss")
p2, = host.plot(range(epochs), AE_list, 'k-', lw=2, label="AE-loss")
p3, = par.plot(range(epochs), np.array(pairwise_list), 'b-', lw=2, label="Pairwise-loss")
leg = plt.legend(loc='best')
host.yaxis.get_label().set_color(p1.get_color())
leg.texts[0].set_color(p1.get_color())
par.yaxis.get_label().set_color(p3.get_color())
leg.texts[2].set_color(p3.get_color())
plt.show()

In [0]:
plt.plot( nmi_train1, 'r-', lw=2, label='nmi_train1')
plt.plot( nmi_train2, 'y-', lw=2, label='nmi_train2')
plt.plot( nmi_train3, 'g-', lw=2, label='nmi_train3')
plt.plot( nmi_val1, 'r--', lw=2, label='nmi_val1')
plt.plot( nmi_val2, 'y--', lw=2, label='nmi_val2')
plt.plot( nmi_val3, 'g--', lw=2, label='nmi_val3')
# plt.plot(range(epochs), np.array(pairwise_list)*lamd, 'b-', lw=3, label='pairwise_loss')
plt.legend(loc='upper left');
plt.show()